In [297]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
import matplotlib.pyplot as plt 
import time
import numpy as np
import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
from nltk.corpus import stopwords
s=stopwords.words('english') + ['']

# Machine learning
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [227]:
query = session.query(Article).filter(or_(Article.journal_ref.like('Physics Review Letters%'),
                                          Article.journal_ref.like('Phys. Rev. Lett.%'),
                                          Article.journal_ref.like('PRL%')))
abstract_all = [x.abstract for x in query.all()]
# titleNatureScience = [x.title for x in query.all()]
session.close_all()

In [357]:
abstract_train = abstract_all
print len(abstract_train)

14390


CountVectorizer would do a cleaner job.

'data' is a list of tuples, where each entry is an n-gram followed by a single word that followed it in some article. 

This should really be hashed as a dict of dict of dict... ngram times. The first dict searches (hashes!) the first word, which returns a dict by which one searches for the next word, etc.

In [358]:
ngram = 4
data_list = []
# re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi

start = time.time()
# Remove punctuation. Later I'd like to keep periods. Also use lowercase only.
pattern1 = re.compile("[^a-zA-Z.]")
# The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
pattern2 = re.compile("(\s{2,})")

for item in abstract_train:
    tmp = pattern2.sub(" ", pattern1.sub(" ", item.lower().lstrip().rstrip()))
    textlist = tmp.split(' ')
    ind = 0
    while ind + ngram - 1 < len(textlist):
        currentTuple = tuple(textlist[ind:(ind+ngram-1)])
        nextWord = textlist[ind+ngram-1]
#         if currentTuple in data.keys():
#             data[currentTuple].append(nextWord)
#         else:
#             data[currentTuple] = [nextWord]
        data_list.append((tuple(textlist[ind:(ind+ngram-1)]), textlist[ind+ngram-1]))
        
        ind += 1
data_list.sort()
print time.time() - start

12.0043869019


In [360]:
start = time.time()

# The list is sorted, so each subsuquent entry is either new or a continuation of the previous one.
data = dict()
previousKey = None
for val in data_list:
    currentKey = val[0]
    if currentKey == previousKey:
        data[currentKey].append(val[1])
    else:
        data[currentKey] = [val[1]]
    previousKey = val[0]

print time.time() - start

5.40404415131


Now I have to search data. If I were smart, I'd make data into an object and encode search functions. I'm not smart.

In [361]:
def find_next_word(ngram):
    '''ngram is a list of (typ. 3) words. Return the options for the next word.'''
    # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
    if ngram in data.keys():
        solution = data[ngram]
        # If there are more than one result, I should either return the most popular option or
        # a random answer weighted by popularity.
        count = Counter(solution)
        return count.most_common(1)[0][0]
    else:
        return None
    
def write_abstract(beginning):
    fake_abstract = list(beginning)
    nextWord = ''
    ind = len(beginning) - ngram + 1
    while (nextWord is not None) and (ind < 200):
#         print fake_abstract[ind:(ind+ngram)]
        nextWord = find_next_word(tuple(fake_abstract[ind:(ind+ngram)]))
#         print nextWord
        if nextWord is None:
            break
        else:
            fake_abstract.append(nextWord)
            ind += 1
    return ' '.join(fake_abstract)
    
# print find_ngram[('a', 'model', 'of')]
# print find_next_word(('a', 'function', 'of'))

In [375]:
print find_next_word(('here','we', 'observe'))

such


In [376]:
# 4 grams: take 3 words and predict the 4th
start = time.time()
abstract1a = write_abstract(['a', 'function', 'of', 'the'])
abstract2a = write_abstract(['here','we', 'observe', 'the'])
abstract3a = write_abstract(['we','measure', 'the', 'mass'])
abstract4a = write_abstract(['in','this', 'letter', 'we'])
print abstract1a
print ''
print abstract2a
print ''
print abstract3a
print ''
print abstract4a
print time.time() - start

a function of the interaction strength and the cavity is strongly excited within fs demonstrating that the fit is consistent with the standard model higgs boson is a scalar or a pseudoscalar particle.

here we observe the formation of a spin density wave sdw order in a wide range of parameters and both transition lines continue into the superconducting state of the art experimental setups and thus has immediate applications for quantum information processing and quantum simulation with alkaline earth atoms. we show that the spin orbit coupling in the electronic structure of the material and represents the number of atoms in a one dimensional optical lattice. we show that the spin orbit coupling in the electronic structure of the material and represents the number of atoms in a one dimensional optical lattice. we show that the spin orbit coupling in the electronic structure of the material and represents the number of atoms in a one dimensional optical lattice. we show that the spin orb

In [314]:
# 5 grams: take 4 words and predict the 5th
abstract1b = write_abstract(['a', 'function', 'of', 'the'])
abstract2b = write_abstract(['here','we', 'observe', 'the'])
abstract3b = write_abstract(['we','measure', 'the', 'mass'])
abstract4b = write_abstract(['in','this', 'letter', 'we'])
print abstract1b
print ''
print abstract2b
print ''
print abstract3b
print ''
print abstract4b

a function of the interaction strength and temperature. at low temperature thermal fluctuations induce magnetic long range order of the eu f moments.

here we observe the phenomenon of sudden transition of quantum correlations when local phase and amplitude damping channels are applied to the case of a single impurity immersed in a fermi sea close to an interspecies p wave feshbach resonance. we show that the spin orbit interaction and the crossing of the critical point. we also make a quantitative prediction on increasing the upper critical field hc and the high field response resembles that of the paramagnetic normal state.

we measure the mass of the leggett mode is small close to the critical point of the case with a large zeeman magnetic field. our scenario provides a promising approach for quantum state transfer to mesoscopic systems using edge states as a quantum channel and show that it is possible to extract a phase diagram for a double quantum well bilayer electron gas in the

In [317]:
# 6 grams: take 5 words and predict the 6th
abstract1c = write_abstract(['a', 'function', 'of', 'the', 'interaction'])
abstract2c = write_abstract(['here','we', 'observe', 'the', 'formation'])
abstract3c = write_abstract(['we','measure', 'the', 'mass', 'of'])
abstract4c = write_abstract(['in','this', 'letter', 'we', 'show'])
print abstract1c
print ''
print abstract2c
print ''
print abstract3c
print ''
print abstract4c

a function of the interaction strength and the imbalance parameter in two and three spatial dimensions. we find that these results are interpreted in terms of a dynamically evolving texture of localized growth modes with superimposed diffusive modes. applying derrick s theorem it is conjectured that the upper critical dimension is infinity. level statistics are investigated in detail. we show that the two dimensional voter model usually considered to only be a marginal coarsening system represents a broad class of models of decoherence including the photon environment we use to obtain most of our information. this leads to objective reality via the presence of robust

here we observe the formation

we measure the mass of the top quark pair with and without the additional jet and for the transverse momentum and the polarization of the fields which act as ancilla systems. information about the spatial correlations is obtained via measurements on the ancilla qubits. among other applicatio

In [321]:
for item in abstract_train:
    if 'natural line width' in item:
        print item
print ''

for item in abstract_train:
    if 'minimize the ohmic' in item.lower():
        print item
print ''

for item in abstract_train:
    if 'mass of the top quark pair' in item.lower():
        print item
print ''

for item in abstract_train:
    if 'here we observe' in item.lower():
        print item
print ''



  We measure the mass difference, \Delta m_0, between the D*(2010)+ and the D0
and the natural line width, \Gamma, of the transition D*(2010)+ -> D0 pi+. The
data were recorded with the BABAR detector at center-of-mass energies at and
near the Upsilon(4S) resonance, and correspond to an integrated luminosity of
approximately 477 1/fb. The D0 is reconstructed in the decay modes D0 -> K- pi+
and D0 -> K- pi+ pi- pi+. For the decay mode D0 -> K- pi+ we obtain Gamma =
(83.4 +- 1.7 +- 1.5) keV and \Delta m_0 = (145 425.6 +- 0.6 +- 1.8) keV, where
the quoted errors are statistical and systematic, respectively. For the D0 ->
K- pi+ pi- pi+ mode we obtain \Gamma = (83.2 +- 1.5 +- 2.6) keV and \Delta m_0
= (145 426.6 +- 0.5 +- 2.0) keV. The combined measurements yield \Gamma = (83.3
+- 1.2 +- 1.4) keV and \Delta m_0 = (145 425.9 +- 0.4 +- 1.7) keV; the width is
a factor of approximately 12 times more precise than the previous value, while
the mass difference is a factor of approximately 6 times